In [3]:
import sys
from pathlib import Path


def sys_info():
    frmt = '\nPython ver: {}\nPython env: {}\n'
    frmt += 'OS:         {}\nCurrent dir: {}\n'
    print(frmt.format(sys.version, 
                      Path(sys.prefix).name,
                      sys.platform,
                      Path.cwd()))
    
def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    if up:
        newp = Path(this_path).parent
    else:
        newp = Path(this_path)
        
    src = newp.joinpath("src")
    if src.exists():
        newp = str(src)
    else:
        newp = str(newp)
        
    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))

# if notebook inside another folder, eg ./notebooks:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), Path.cwd().name.startswith(nb_folder))


def get_project_dirs(which=['data', 'images'], nb_folder='notebooks'):
    dir_lst = []
    if Path.cwd().name.startswith(nb_folder):
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

DIR_DATA, DIR_IMG = get_project_dirs()
    
import numpy as np
import scipy as sp
from scipy import stats as sps
import pandas as pd
#pd.set_option("display.max_colwidth", 200)

import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sb
#plt.ion()
%matplotlib inline
plt.style.use('seaborn-v0_8-muted')

from pprint import pprint as pp

# Filtered dir() for method discovery:
def fdir(obj, start_with_str='_', exclude=True):
    return [d for d in dir(obj) if not d.startswith(start_with_str) == exclude]


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import HTML, Markdown #, IFrame
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))

# autoreload extension
from IPython import get_ipython
ipython = get_ipython()

def show_versions():
    txt = '<pre><br>'
    txt += F'Python:\t\t{sys.version}<br>'
    txt += F'Python env:\t{Path(sys.prefix).name}<br>'
    txt += F'Numpy:\t\t{np.__version__}<br>'
    txt += F'Scipy:\t\t{sp.__version__}<br>'
    txt += F'Pandas:\t\t{pd.__version__}<br>'
    txt += F'Matplotlib:\t{mpl.__version__}<br>'
    txt += F'Currrent dir: {Path.cwd()}'
    txt += '</pre>'
    div = f"""<div class="alert alert-info"><b>Versions:</b><br>{txt}</div>"""
    return HTML(div)


if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2
#..................

sys_info()

no_wmark = False
try:
    %load_ext watermark
    %watermark
except ModuleNotFoundError:
    no_wmark = True

if no_wmark:
    show_versions()
else:
    %watermark -iv


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Python ver: 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:16:33) [MSC v.1929 64 bit (AMD64)]
Python env: ds310
OS:         win32
Current dir: C:\Users\catch\Documents\GH_Projects\Pythonics\notebooks

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: 2023-09-19T15:44:36.714338-04:00

Python implementation: CPython
Python version       : 3.10.8
IPython version      : 8.8.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
CPU cores   : 8
Architecture: 64bit

sys       : 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:16:33) [MSC v.1929 64 bit (AMD64)]
numpy     : 1.23.5
matplotlib: 3.6.3
seaborn   : 0.12.2
pandas    : 1.5.3
re        : 2.2.1
scipy     : 1.9.3



---
---
# Manipulations using the `re` module
---

In [4]:
DIR_DATA

WindowsPath('C:/Users/catch/Documents/GH_Projects/Pythonics/data')

In [111]:
import re

runprm_file = DIR_DATA.joinpath('run.prm.qq')

def load_runprm(filename):
        """Process run.prm* file for populating dictionary self.runprm.
        """
        # Create a regex that matches lines ending with ')'
        pattern = r".*\)$"

        prm_d = dict()
        # Sample line in run.prm:
        # "t        step 1: pre-run, pdb-> mcce pdb                    (DO_PREMCCE)"
        with open(filename) as f:
            for line in f:
                line = line.strip()
                
                # skip "MCCE CONFIGURATION FILE (date)" line, comments and blanks
                if not line:
                    continue
                if line.startswith("MCCE"):
                    continue
                if line.startswith("#"):
                    continue

                if not re.match(pattern, line):
                    continue
                    
                left_p = line.rfind("(")
                if left_p <= 0:
                    # not found or at 0:
                    continue

                key = line[left_p + 1:-1]
                if key.islower():
                    # not a key: comment in parentheses?
                    continue
                fields = line[:left_p].split()
                if fields:
                    value = fields[0]
                else:
                    continue

                prm_d[key] = value
                
        return prm_d

In [113]:
prm_d = load_runprm(runprm_file)
prm_d

{'INPDB': 'xyz.pdb',
 'DO_PREMCCE': 'f',
 'DO_ROTAMERS': 'f',
 'DO_ENERGY': 'f',
 'DO_MONTE': 'f',
 'SIDECHAIN_OPT': '0',
 'PBE_SOLVER': 'delphi',
 'RXN_METHOD': 'surface',
 'EPSILON_PROT': '4.0',
 'EXTRA': '/home/mcce/mcce2.5.1/extra.tpl',
 'RENAME_RULES': '/home/mcce/mcce2.5.1/name.txt',
 'TITR_TYPE': 'ph',
 'TITR_PH0': '0.0',
 'TITR_PHD': '1.0',
 'TITR_EH0': '0.0',
 'TITR_EHD': '30.0',
 'TITR_STEPS': '15',
 'MCCE_HOME': '/home/mcce/mcce2.5.1',
 'MINIMIZE_SIZE': 't',
 'DO_TRACE': 't',
 'TERMINALS': 't',
 'CLASH_DISTANCE': '2.0',
 'H2O_SASCUTOFF': '-0.05',
 'IGNORE_INPUT_H': 't',
 'ROT_SPECIF': 'f',
 'ROT_SWAP': 't',
 'PACK': 'f',
 'ROTATIONS': '6',
 'SAS_CUTOFF': '1.00',
 'VDW_CUTOFF': '10.0',
 'REPACKS': '5000',
 'REPACK_CUTOFF': '0.01',
 'HDIRECTED': 'f',
 'HDIRDIFF': '1.0',
 'HDIRLIMT': '36',
 'HV_RELAX_NCYCLE': '0',
 'HV_RELAX_DT': '4',
 'HV_RELAX_NITER': '100',
 'HV_RELAX_VDW_THR': '2.',
 'HV_RELAX_HV_VDW_THR': '10.',
 'HV_TORS_SCALE': '20.',
 'HV_RELAX_N_SHAKE': '10000',
 'HV_R

In [141]:
tpl_file = DIR_DATA.joinpath('ala.ftpl')
tpl_extra = DIR_DATA.joinpath('extra.tpl')


float_values = ["EPSILON_PROT", "TITR_PH0", "TITR_PHD", "TITR_EH0", "TITR_EHD", "CLASH_DISTANCE",
                "BIG_PAIRWISE", "MONTE_T", "MONTE_REDUCE", "EXTRAE", "SCALING"]
int_values = ["TITR_STEPS", "MONTE_RUNS", "MONTE_TRACE", "MONTE_NITER", "MONTE_NEQ",
              "MONTE_NSTART", "MONTE_FLIPS"]

def print_dict(d):
    for k, v in d.items():
        print(f"{k:25}:{v}")


def load_tpl(fname):
    """Load a tpl file."""
    tpl = dict()
    
    with open(fname) as f:
        for line in f:
            line = line.split("#")[0]
            if not line:
                continue

            fields = line.split(":")
            if len(fields) != 2:
                continue
            
            key_string = fields[0].strip()
            keys = key_string.split(",")
            keys = [x.strip().strip("\"") for x in keys]
            keys = tuple([x for x in keys if x])

            value_string = fields[1].strip()
            if keys[0] in float_values:
                tpl[keys] = float(value_string)
            elif keys[0] in int_values:
                tpl[keys] = int(value_string)
            else:
                tpl[keys] = value_string

    return tpl


def print_tpl(tpl):
    print_dict(tpl)


def read_extra(tpl_ex):
    """Read extra.tpl."""
    #self.load_tpl(self.runprm["EXTRA"])
    d = load_tpl(tpl_file)
    default_values_keys = [("SCALING", "VDW0"),
                           ("SCALING", "VDW1"),
                           ("SCALING", "VDW"),
                           ("SCALING", "TORS"),
                           ("SCALING", "ELE"),
                           ("SCALING", "DSOLV")]
    for element in default_values_keys:
        #if element not in self.tpl:
        if element not in d:
            d[element] = 1.0

    return d

SCALING_FACTORS = [("SCALING", "VDW0"),
                   ("SCALING", "VDW1"),
                   ("SCALING", "VDW"),
                   ("SCALING", "TORS"),
                   ("SCALING", "ELE"),
                   ("SCALING", "DSOLV")]

def print_scaling(tpl: dict):
    """Print scaling factors."""
    # print self.param
    print("   Scaling factors:")
    print("   VDW0  = %.3f" % tpl[("SCALING", "VDW0")])
    print("   VDW1  = %.3f" % tpl[("SCALING", "VDW1")])
    print("   VDW   = %.3f" % tpl[("SCALING", "VDW")])
    print("   TORS  = %.3f" % tpl[("SCALING", "TORS")])
    print("   ELE   = %.3f" % tpl[("SCALING", "ELE")])
    print("   DSOLV = %.3f" % tpl[("SCALING", "DSOLV")])
    print("   Done\n")


In [148]:


#out = f"\tVDW0  = {:.3f}\n\tVDW1  = {:.3f}\n\tVDW   = {.3f}\n\tTORS  = {.3f}\n\tELE   = {:.3f}\n\tDSOLV = {:.3f}\n\tDone\n"

out = """\tScaling factors:
\tVDW0  = {:.3f}
\tVDW1  = {:.3f}
\tVDW   = {:.3f}
\tTORS  = {:.3f}
\tELE   = {:.3f}
\tDSOLV = {:.3f}
\tDone\n"""
print(out.format(*[extratpl_d[sf] for sf in SCALING_FACTORS]))

	VDW0  = 1.000
	VDW1  = 1.000
	VDW   = 1.000
	TORS  = 1.000
	ELE   = 1.000
	DSOLV = 1.000
	Done



In [140]:
print_scaling(extratpl_d)

   Scaling factors:
   VDW0  = 1.000
   VDW1  = 1.000
   VDW   = 1.000
   TORS  = 1.000
   ELE   = 1.000
   DSOLV = 1.000
   Done



In [138]:
extratpl_d = read_extra(tpl_extra)

extratpl_d 

{('CONFLIST', 'ALA'): 'ALABK, ALA01',
 ('CONNECT', ' N  ', 'ALABK'): 'sp2, " ?  ", " CA ", " H  "',
 ('CONNECT', ' H  ', 'ALABK'): 's, " N  "',
 ('CONNECT', ' CA ', 'ALABK'): 'sp3, " N  ", " C  ", " CB ", " HA "',
 ('CONNECT', ' HA ', 'ALABK'): 's, " CA "',
 ('CONNECT', ' C  ', 'ALABK'): 'sp2, " CA ", " O  ", " ?  "',
 ('CONNECT', ' O  ', 'ALABK'): 'sp2, " C  "',
 ('CONNECT', ' CB ', 'ALA01'): 'sp3, " CA ", " HB1", " HB2", " HB3"',
 ('CONNECT', ' HB1', 'ALA01'): 's, " CB "',
 ('CONNECT', ' HB2', 'ALA01'): 's, " CB "',
 ('CONNECT', ' HB3', 'ALA01'): 's, " CB "',
 ('CHARGE', 'ALABK', ' N  '): '-0.350',
 ('CHARGE', 'ALABK', ' H  '): '0.250',
 ('CHARGE', 'ALABK', ' CA '): '0.100',
 ('CHARGE', 'ALABK', ' HA '): '0.000',
 ('CHARGE', 'ALABK', ' C  '): '0.550',
 ('CHARGE', 'ALABK', ' O  '): '-0.550',
 ('CHARGE', 'ALA01', ' CB '): '0.000',
 ('CHARGE', 'ALA01', ' HB1'): '0.000',
 ('CHARGE', 'ALA01', ' HB2'): '0.000',
 ('CHARGE', 'ALA01', ' HB3'): '0.000',
 ('RADIUS', 'ALABK', ' N  '): '1.500,   

In [131]:
line = "#>>>START of original comments, this file was converted from old format"
line = line.split("#")[0]
fields = None
if not line:
    print("empty")
else:
    fields = line.split(":")
line
fields

empty


''

In [132]:
tpl_d = load_tpl(tpl_file)
tpl_d

{('CONFLIST', 'ALA'): 'ALABK, ALA01',
 ('CONNECT', ' N  ', 'ALABK'): 'sp2, " ?  ", " CA ", " H  "',
 ('CONNECT', ' H  ', 'ALABK'): 's, " N  "',
 ('CONNECT', ' CA ', 'ALABK'): 'sp3, " N  ", " C  ", " CB ", " HA "',
 ('CONNECT', ' HA ', 'ALABK'): 's, " CA "',
 ('CONNECT', ' C  ', 'ALABK'): 'sp2, " CA ", " O  ", " ?  "',
 ('CONNECT', ' O  ', 'ALABK'): 'sp2, " C  "',
 ('CONNECT', ' CB ', 'ALA01'): 'sp3, " CA ", " HB1", " HB2", " HB3"',
 ('CONNECT', ' HB1', 'ALA01'): 's, " CB "',
 ('CONNECT', ' HB2', 'ALA01'): 's, " CB "',
 ('CONNECT', ' HB3', 'ALA01'): 's, " CB "',
 ('CHARGE', 'ALABK', ' N  '): '-0.350',
 ('CHARGE', 'ALABK', ' H  '): '0.250',
 ('CHARGE', 'ALABK', ' CA '): '0.100',
 ('CHARGE', 'ALABK', ' HA '): '0.000',
 ('CHARGE', 'ALABK', ' C  '): '0.550',
 ('CHARGE', 'ALABK', ' O  '): '-0.550',
 ('CHARGE', 'ALA01', ' CB '): '0.000',
 ('CHARGE', 'ALA01', ' HB1'): '0.000',
 ('CHARGE', 'ALA01', ' HB2'): '0.000',
 ('CHARGE', 'ALA01', ' HB3'): '0.000',
 ('RADIUS', 'ALABK', ' N  '): '1.500,   

In [78]:
line = "(Create a regular expression that matches lines ending with a closed parenthesis"
pattern = r".*\)$"

left_p = 0
fields = line[:left_p].split()
print("fields:", fields, len(fields))

if fields:
    value = fields[0]
    print("val", value)
else:
    print(" fields is F")
    
if len(fields) < 1:
    print("<1")
else:
    value = fields[0]
    
if not re.match(pattern, line):
    print("no match")
else:
    print(line)

fields: [] 0
 fields is F
<1
no match
